In [1]:
class MersenneTwister(object):
    # First start with constants
    w, n, m, r = 32, 624, 397, 31
    a = 0x9908B0DF
    u, d = 11, 0xFFFFFFFF
    s, b = 7, 0x9D2C5680
    t, c = 15, 0xEFC60000
    l = 18
    f = 1812433253
    
    mt = [0] * n

    index = n+1
    lower_mask: int = (1 << r) - 1
    upper_mask: int = (~lower_mask) & sum( 0b1 << shift for shift in range(w) )
    
    def __init__(self, seed: int):
        mt = self.mt # just so I don't have to write self a bunch of times

        self.index = self.n
        mt[0] = seed
        for i in range(1, self.n):
            mt[i] = sum( 0b1 << shift for shift in range(self.w) ) & ( self.f * ( mt[i-1] ^ ( mt[i-1] >> (self.w-2) )) + 1 )
            
    def __twist__(self):
        mt = self.mt
        for i in range(self.n):
            x = ( mt[i] & self.upper_mask ) | ( mt[ (i+1) % self.n ] & self.lower_mask )
            xA = x >> 1
            if x % 2 != 0:
                xA ^= self.a
            mt[i] = mt[ (i+self.m) % self.n ] ^ xA
        self.index = 0
        
    def __call__(self):
        mt = self.mt
        
        if self.index >= self.n:
            if self.index > self.n:
                assert False, "Generator was never seeded"
            else:
                self.__twist__()
        
        y  =  mt[self.index]
        y ^= (y >> self.u) & self.d
        y ^= (y << self.s) & self.b
        y ^= (y << self.t) & self.c
        y ^= (y >> self.l)
    
        self.index += 1
    
        return y & sum( 0b1 << shift for shift in range(self.w) )
        

In [2]:
from time import time, sleep
from random import random as RandFloat

sleep( RandFloat() * 100 )
__seed__ = int(time())
sleep( RandFloat() * 100 )
mt = MersenneTwister(__seed__)

In [3]:
first_number = mt()
first_number

153326920

In [4]:
toc = int(time())

# I'm just going to brute force this.
# I can't think of any other way and I've seen a bunch of other people use that as a solution.

for offset in range(1000):
    mt2 = MersenneTwister(toc - offset)
    if mt2() == first_number:
        print(f'Seed was {toc-offset}')
        break

Seed was 1670776217


In [5]:
print(__seed__)

1670776217
